# Keras and MNIST, HRNN

##### From: https://github.com/keras-team/keras/blob/master/examples/mnist_hierarchical_rnn.py

In [30]:
#Import necessary libraries
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, Input, LSTM
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Model

In [14]:
# Set training variables
batch_size = 32
num_classes = 10 
epochs = 5

In [15]:
# Enter rows and columns of image
row_hidden = 128
col_hidden = 128

In [16]:
# Split into training and testing data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### Step 0: Get Data into Useable Form

In [26]:
# Need to shape to 4D for HRNN

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_train = x_train / 255

x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_test = x_test.astype('float32')
x_test = x_test / 255

print('Shape of x_train:', x_train.shape)
print('Number of Training Samples:', x_train.shape[0])
print('Number of Testing Samples: ', x_test.shape[0])

Shape of x_train: (60000, 28, 28, 1)
Number of Training Samples: 60000
Number of Testing Samples:  10000


In [18]:
# Output y is a normal list of values
print(y_train[:2])

[5 0]


In [19]:
# Convert y values to one-hot-encoded values
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [20]:
# Output y is now one-hot-encoded
print(y_train[:2])

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [22]:
# Define x_train values
print(x_train.shape[1:])
row, col, pixel = x_train.shape[1:]

(28, 28, 1)


In [23]:
# Define the input
x = Input(shape=(row, col, pixel))
print(x)

Tensor("input_1:0", shape=(?, 28, 28, 1), dtype=float32)


In [24]:
# Encode a row of pixels
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

In [25]:
# Encode a column of encoded rows
encoded_columns = LSTM(col_hidden)(encoded_rows)

### Step 1: Define Network

In [28]:
# Make predictions
prediction = Dense(num_classes, activation='softmax')(encoded_columns)

In [31]:
# Create model using prediction, not sequence
model = Model(x, prediction)

### Step 2: Compile Network

In [32]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

### Step 3: Fit Network

In [33]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 248s 4ms/step - loss: 1.6879 - acc: 0.3749 - val_loss: 0.6649 - val_acc: 0.7817
Epoch 2/5
60000/60000 [==============================] - 241s 4ms/step - loss: 0.3905 - acc: 0.8754 - val_loss: 0.3375 - val_acc: 0.8905
Epoch 3/5
60000/60000 [==============================] - 269s 4ms/step - loss: 0.1964 - acc: 0.9392 - val_loss: 0.1696 - val_acc: 0.9458
Epoch 4/5
60000/60000 [==============================] - 295s 5ms/step - loss: 0.1374 - acc: 0.9572 - val_loss: 0.1367 - val_acc: 0.9570
Epoch 5/5
60000/60000 [==============================] - 289s 5ms/step - loss: 0.1059 - acc: 0.9671 - val_loss: 0.0996 - val_acc: 0.9684


### Step 4: Evaluate Network

In [34]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Model Accuracy: ", score[1])

Model Accuracy:  0.9684
